In [2]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from ta.trend import SMAIndicator
# import pandas_datareader as pd_web
import yfinance as yf
import pandas as pd
# import mplfinance as mpf
import json

import numpy as np

class khalman:
    def __init__(self,new_info=0.5,old_info=0.5):
        self.y_n=1
        self.y_n1=1
        self.new_info = new_info
        self.old_info = old_info
    def filter(self,x_0):
        temp_val = self.y_n
        self.y_n = self.new_info*x_0 + self.old_info*self.y_n1
        self.y_n1 = temp_val
        return self.y_n

class derivative_prom:
    def __init__(self,m,n,o,p,q):
        self.x4=1
        self.x3=1
        self.x2=1
        self.x1=1
        self.x=1
        temp_total= m+n+o+p+q
        self.m=m/temp_total
        self.n=n/temp_total
        self.o=o/temp_total
        self.p=p/temp_total
        self.q=q/temp_total
    def filter(self,x0):
        temp_value = self.m*(x0-self.x)+self.n*(x0-self.x1)+self.o*(x0-self.x2)+self.p*(x0-self.x3)+self.q*(x0-self.x4) 
        self.x4=self.x3
        self.x3=self.x2
        self.x2=self.x1
        self.x1=self.x
        self.x=x0
        return temp_value


def cagr(df:pd.DataFrame):
    temp_df= df.copy()
    temp_df['retorno_diario'] = temp_df['Close'].pct_change()
    temp_df['retorno_acumulado'] = (1+ temp_df['retorno_diario']).cumprod()
    numero_de_anios=temp_df.shape[0]/252
    return (temp_df['retorno_acumulado'][-1])**(1/numero_de_anios)-1 #### CAGR

def volatilidad(df:pd.DataFrame):
    temp_df = df.copy()
    temp_df['retorno_diario']=temp_df['Close'].pct_change()
    return temp_df['retorno_diario'].std()*np.sqrt(252) #### VOLATILIDAD

def sharpe(df:pd.DataFrame,riskfree_rate=0.022):
    temp_df = df.copy()
    return (cagr(df)-riskfree_rate)/volatilidad(df) #### SHARPE RATIO

def sortino(df:pd.DataFrame,riskfree_rate=0.022):
    temp_df = df.copy()
    temp_df['retorno_diario'] = temp_df['Close'].pct_change()
    neg_vol = temp_df[ temp_df['retorno_diario']<0 ]['retorno_diario'].std()*np.sqrt(252)
    return (cagr(temp_df)-riskfree_rate)/neg_vol #### sortino RATIO

def MDD(df:pd.DataFrame):
    temp_df = df.copy()
    temp_df['retorno_diario'] = temp_df['Close'].pct_change()
    temp_df['retorno_acumulativo']  = (1 + temp_df['retorno_diario']).cumprod()
    temp_df['max_ret_acum'] = temp_df['retorno_acumulativo'].cummax()
    temp_df['drawdown']= temp_df['max_ret_acum']-temp_df['retorno_acumulativo']
    temp_df['drawdown_pct']= temp_df['drawdown']/temp_df['max_ret_acum']
    return temp_df['drawdown_pct'].max()

def calmar(df:pd.DataFrame):
    temp_df = df.copy()
    return cagr(df)/MDD(df)

def get_meashures_n(df:pd.DataFrame):
    CAGR = cagr(data)
    VOL = volatilidad(data)
    SHARP = sharpe(data)
    SORTI = sortino(data)
    M = MDD(data)
    CAL = calmar(data)
    print("----------- data natural meashures -------------")
    print(f"CAGR :-> %{CAGR*100}\t\trentabilidad anual")
    print(f"VOL  :-> %{VOL*100}\t\tvolatilidad anual")
    print(f"SHARP:-> {SHARP}\t\t(R>1?)")
    print(f"SORTI:-> {SORTI}\t\t(R>1?)")
    print(f"MDD  :-> %{M*100}\t\tMáxima caida relativa")
    print(f"CALM :-> {CAL}\t\trentabilidad/riezgo")
    print("----------- data natural meashures -------------\n----\n--\n-")

    return { 'cagr':CAGR,'volatility':VOL,'sharpe':SHARP,'sortino':SORTI,'mdd':M,'calmar':CAL }

def strategy_sma_implementation(df:pd.DataFrame,low,high):
    temp_df = df.copy()
    sma_L = SMAIndicator(data['Close'],low,True)
    sma_H = SMAIndicator(data['Close'],high,True)
    temp_df['sma-L']= sma_L.sma_indicator()
    temp_df['sma-H']= sma_H.sma_indicator()
    temp_df['pos']=np.where(temp_df['sma-L']>temp_df['sma-H'],1,-1)
    plot_many(temp_df,"Close","sma-L","sma-H",'pos')
    return temp_df['pos']

def strategy_advanced(df:pd.DataFrame,newd=0.5,oldd=0.5):
    temp_df = df.copy()
    temp_total= newd+oldd
    filter_L = khalman(newd/temp_total,oldd/temp_total)
    temp_df['klm-L']=temp_df['Close'].apply(filter_L.filter)
    # temp_df['pos']=temp_df['klm-L'].apply(lambda x: 1 if x>0 else -1)
    temp_df['pos']=np.where(temp_df['Close']>temp_df['klm-L'],1,-1)
    plot_many(temp_df,"Close","klm-L","pos")
    # return temp_df['pos'].apply(lambda x: 1 if x>0 else -1)
    # return temp_df['pos'].apply(lambda x: 1 if x>0 else -1)
    return np.where(temp_df['Close']>temp_df['klm-L'],1,-1)
    
def plot_many(df:pd.DataFrame,*param):
    temp_df = df.copy()
    fig = go.Figure()
    for item in param:
        fig.add_trace(go.Scatter(x=temp_df.index,y=temp_df[item],name=item))
    fig.show()

def estrategia(df:pd.DataFrame,method='sma',low=15,high=30,plot_graphs=True,no_meashure=False):
    temp_df = df.copy()

    #### strategy implementation
    if(method=='sma'):
        temp_df['posicion']= strategy_sma_implementation(temp_df,low,high)
    elif(method=='klm'):
        temp_df['posicion']= strategy_advanced(temp_df,low,high)
    else:
        temp_df['posicion']= strategy_sma_implementation(temp_df,low,high)

    #### benefit estimation
    # if(plot_graphs):
    #     plot_many(temp_df,"Close")
    temp_df['retornos_del_instrumento'] = temp_df['Close'].pct_change()
    temp_df['estrategia'] = temp_df['posicion'].shift(1)*temp_df['retornos_del_instrumento']
    temp_df['ret_acum'] = temp_df['retornos_del_instrumento'].cumsum().apply(np.exp)
    temp_df['ret_acum_est'] = temp_df['estrategia'].cumsum().apply(np.exp)

    #### meashures
    if(not no_meashure):
        if(plot_graphs):
            plot_many(temp_df,"ret_acum","ret_acum_est")
        def meashures(df:pd.DataFrame,plot_graphs=plot_graphs):
            meashures={}
            temp_df = df.copy()
            numero_de_anios=df.shape[0]/252
            meashures['cagr']=df['ret_acum_est'][-1]**(1/numero_de_anios)-1 #### CAGR
            meashures['volatility']=df['estrategia'].std()*np.sqrt(252)  #### VOLATILITY
            meashures['sharpe']= (meashures['cagr']-0.022)/meashures['volatility']  #### SHARPE
            negvol= df[df['estrategia']<0]['estrategia'].std()*np.sqrt(252)  #### SORTINO
            meashures['sortino']= (meashures['cagr']-0.022)/negvol  #### SORTINO

            temp_df['max_ret_acum']= temp_df['ret_acum_est'].cummax()
            temp_df['drawdown']= temp_df['max_ret_acum']-temp_df['ret_acum_est']
            temp_df['drawdown_pct']=temp_df['drawdown']/temp_df['max_ret_acum']
            meashures['mdd']= temp_df['drawdown_pct'].max()
            meashures['calmar']=meashures['cagr']/meashures['mdd']

            meashures['porcentaje superado']=(df['ret_acum_est'][df.shape[0]-1]-df['ret_acum'][df.shape[0]-1])/abs(df['ret_acum'][df.shape[0]-1]) #### BETTER PERFORMING PERCENTAGE
            if(plot_graphs):
                print("----------- data strategy meashures -------------")
                print(f"BEST :-> %{meashures['porcentaje superado']*100}\t\tporcentaje de mejoramiento del retorno por estrategia")
                print(f"CAGR :-> %{meashures['cagr']*100}\t\trentabilidad anual")
                print(f"VOL  :-> %{meashures['volatility']*100}\t\tvolatilidad anual")
                print(f"SHARP:->  {meashures['sharpe']}\t\t(R>1?)")
                print(f"SORTI:->  {meashures['sortino']}\t\t(R>1?)")
                print(f"MDD  :-> %{meashures['mdd']*100}\t\tMáxima caida relativa")
                print(f"CALM :->  {meashures['calmar']}\t\trentabilidad/riezgo")
                print("----------- data strategy meashures -------------\n----\n--\n-")
            return meashures
        return meashures(temp_df)
    else:
        meashures={}
        numero_de_anios=df.shape[0]/252
        meashures['cagr']=temp_df['ret_acum_est'][-1]**(1/numero_de_anios)-1 #### CAGR
        return meashures

def get_best_paramethers(df:pd.DataFrame,eval:callable,lowest=50,highest=200,paramether='cagr'):
    best = {'low':0,'high':100}
    last_best = 0
    for low in range(lowest):
        for high in range(highest):
            percentage = low*highest + high
            if((100*percentage/(lowest*highest))%10==0):
                print(f"%{100*percentage/(lowest*highest)}")
            results = estrategia(data['2018-01-01':'2021-02-01'],"klm",low+1,high+1,False,True)
            # print(results)
            vals= results[paramether]
            if( eval(vals,last_best) ):
                last_best = vals
                best = {'low':low,'high':high}
    return best



In [3]:
# with open("data/brands_list.json") as file:
#     assets = json.load(file)['BRANDS']

# for item in assets:
#     # data = yf.download([assets[4]],interval='1d',start='2016-01-01',end='2021-04-30')
#     print(f"################### {item} ####################")
#     data = yf.download(item,interval='1d',start='2016-01-01',end='2021-04-30')
#     data = data.fillna(method='bfill')
#     data = data.fillna(method='ffill')
#     print(item,get_best_paramethers(data['2018-01-01':'2021-02-01'],lambda a,b:a>b,50,200,'cagr'))
#     # cagr_hold = get_meashures_n(data['2018-01-01':'2021-02-01'])

In [8]:
print(f"################### ETC-USD ####################")
data = yf.download("MSFT",interval='1d',start='2020-01-01',end='2021-11-30')
data = data.fillna(method='bfill')
data = data.fillna(method='ffill')
val = estrategia(data['2020-01-01':'2021-11-01'],'sma',7,10,True)
cagr_hold = get_meashures_n(data['2018-01-01':'2021-11-01'])


################### ETC-USD ####################
[*********************100%***********************]  1 of 1 completed


----------- data strategy meashures -------------
BEST :-> %-10.22092718579639		porcentaje de mejoramiento del retorno por estrategia
CAGR :-> %48.39270286733803		rentabilidad anual
VOL  :-> %35.335909460210246		volatilidad anual
SHARP:->  1.3072453369101198		(R>1?)
SORTI:->  1.6329934651318816		(R>1?)
MDD  :-> %29.199231023093997		Máxima caida relativa
CALM :->  1.6573279902153486		rentabilidad/riezgo
----------- data strategy meashures -------------
----
--
-
----------- data natural meashures -------------
CAGR :-> %47.23477417592608		rentabilidad anual
VOL  :-> %34.75048970047398		volatilidad anual
SHARP:-> 1.2959464618799839		(R>1?)
SORTI:-> 1.6749240222275383		(R>1?)
MDD  :-> %28.23529392738187		Máxima caida relativa
CALM :-> 1.6728982633369718		rentabilidad/riezgo
----------- data natural meashures -------------
----
--
-
